<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Planet
[Github Repository](https://github.com/progirep/planet)

In [ ]:
%%capture

# Clone the repo
!git clone https://github.com/progirep/planet.git

In [ ]:
%%capture
# we need a script to convert the neural network description in to the "prototxt" format (produced by Caffe) into a json file
!cd tools
!wget https://gist.github.com/progirep/fd7d2dc120862faa984a70f503611013/raw/260e1e76cebd0ea58bf1a03b64c3f1e0002fc677/csv_to_hdf5_supervised_classification.py 

# we need a second script to generate a database in "HDF5" format from comma-separated value files
!wget https://raw.githubusercontent.com/vadimkantorov/caffemodel2json/3a8fd443bf1596dad5f517aecdef08a81bf73bfe/caffemodel2json.py

In [ ]:
%%capture

# install packages in order to build PLANET
!sudo apt-get install libglpk-dev
!sudo apt-get install qt5-qmake
!sudo apt-get install valgrind
!sudo apt-get install libltdl-dev
!sudo apt-get install protobuf-compiler

In [ ]:
%%capture

# compile the source code
%cd planet/src
%ls
!qmake Tool.pro
!make

In [ ]:
%%capture

# install Caffe and Yices
# change root path of #CAFFE and #YICES
!sudo apt install caffe-cuda
!sudo add-apt-repository ppa:sri-csl/formal-methods -qq
!sudo apt-get update
!sudo apt-get install yices2

In [ ]:
# envs probably useless
%env YICES=/usr
%env CAFFE=/usr
%cd /content/planet/casestudies/MNIST/

In [ ]:
%%bash

export YICES=/usr
export CAFFE=/sr
set -e
caffe train --solver=lenet_solver.prototxt $@ 

Current Problem:
https://github.com/BVLC/caffe/issues/2780
> Check failed: mdb_status == 0



#Current Problems
* csv_to_hdf5_supervised_classificatoin.py doesn't work, because it's Python2
* Error with caffe train - not sure why -https://github.com/BVLC/caffe/issues/2780

#TODO's
* Try to do a local jupyter notebook instead of colab
* Continue the "Running the Collision Avoidance Benchmarks" Section



## Install Caffe

Following [this tutorial](https://colab.research.google.com/github/Huxwell/caffe-colab/blob/main/caffe_details.ipynb). Caution: this takes 5 minutes.

In [ ]:
%cd /content/
!git clone https://github.com/BVLC/caffe.git

In [ ]:
%%capture 
!sudo apt-get install libgflags2.2 
!sudo apt-get install libgflags-dev
!sudo apt-get install libgoogle-glog-dev
!sudo apt-get install libhdf5-100
!sudo apt-get install libhdf5-serial-dev
!sudo apt-get install libhdf5-dev
!sudo apt-get install libhdf5-cpp-100
!sudo apt-get install libprotobuf-dev protobuf-compiler

In [ ]:
!find /usr -iname "*hdf5.so"
# got: /usr/lib/x86_64-linux-gnu/hdf5/serial
!find /usr -iname "*hdf5_hl.so"

In [ ]:
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial.so /usr/lib/x86_64-linux-gnu/libhdf5.so
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial_hl.so /usr/lib/x86_64-linux-gnu/libhdf5_hl.so

In [ ]:
#!find /usr -iname "*hdf5.h*" # got:
# /usr/include/hdf5/serial/hdf5.h 
# /usr/include/opencv2/flann/hdf5.h
# Let's try the first one.
%env CPATH="/usr/include/hdf5/serial/"
#fatal error: hdf5.h: No such file or directory

In [ ]:
%%capture
!sudo apt-get install libleveldb-dev
!sudo apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev
!sudo apt-get install libsnappy-dev

Build caffe from source files.

In [ ]:
!echo $CPATH

In [ ]:
%cd caffe

!ls
!make clean
!cp Makefile.config.example Makefile.config

In [ ]:
!sed -i 's/-gencode arch=compute_20/#-gencode arch=compute_20/' Makefile.config #old cuda versions won't compile 
!sed -i 's/\/usr\/local\/include/\/usr\/local\/include \/usr\/include\/hdf5\/serial\//'  Makefile.config #one of the 4 things needed to fix hdf5 issues
!sed -i 's/# OPENCV_VERSION := 3/OPENCV_VERSION := 3/' Makefile.config #We actually use opencv 4.1.2, but it's similar enough to opencv 3.
!sed -i 's/code=compute_61/code=compute_61 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_75,code=sm_75 -gencode=arch=compute_75,code=compute_75/' Makefile.config #support for new GPUs

In [ ]:
!make all -j 4 # -j would use all availiable cores, but RAM related errors occur

In [ ]:
!find / -iname "*caffe*"

## Train Caffe model on MNIST

In [ ]:
# downloads mnist dataset

%cd /content/caffe/

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz
!cp -rv MNIST/raw/* data/mnist/

In [ ]:
# creates mnist_test_lmdb and mnist_train_lmdb

!/content/caffe/examples/mnist/create_mnist.sh

In [ ]:
# copy lmdbs to planet folder

%cp -a /content/caffe/examples/mnist/mnist_test_lmdb /content/planet/casestudies/MNIST/
%cp -a /content/caffe/examples/mnist/mnist_train_lmdb /content/planet/casestudies/MNIST/

In [ ]:
%cd /content/planet/casestudies/MNIST/

# change iterations from 200k to 50k
!sed -i 's/200000/50000/g' /content/planet/casestudies/MNIST/lenet_solver.prototxt
!sed -i 's/200000/50000/g' /content/planet/casestudies/MNIST/lenet_solver.prototxt

# remove snapshots
!sed -i 's/snapshot:/#snapshot:/g' /content/planet/casestudies/MNIST/lenet_solver.prototxt
!sed -i 's/snapshot_prefix:/#snapshot_prefix:/g' /content/planet/casestudies/MNIST/lenet_solver.prototxt

# train the model
!/content/caffe/build/tools/caffe train --solver=/content/planet/casestudies/MNIST/lenet_solver.prototxt $@

done: training of the model

result: lenet_iter_50000.caffemodel and lenet_iter_50000.solverstate

next step: convert caffemodel file to .rlv so PLANET can verify it
